In [17]:
import torch
import os
from tqdm.auto import tqdm
from transformers import bertaTokenizer,AutoModelForMaskedLM

ImportError: cannot import name 'bertaTokenizer' from 'transformers' (c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py)

In [18]:
from transformers import BertTokenizer

In [20]:
# Load the tokenizer

tokenizer = BertTokenizer.from_pretrained('C:/Bangla_dude/new_ban_t')


In [21]:
print(tokenizer.tokenize("আমি বাংলায় গান গাই।"))

['আমি', 'বাংলায', 'গান', 'গাই', '।']


train code

In [3]:
# Define the file path to save the tensors
file_path = 'bangla_tokens'

In [4]:
# Load the tensors
input_ids = torch.load(os.path.join(file_path, "input_ids.pt"))
mask = torch.load(os.path.join(file_path, "mask.pt"))
labels = torch.load(os.path.join(file_path, "labels.pt"))

In [5]:

input_ids.size(), mask.size(), labels.size()

(torch.Size([1114481, 512]),
 torch.Size([1114481, 512]),
 torch.Size([1114481, 512]))

In [6]:
input_ids.shape

torch.Size([1114481, 512])

In [7]:
mask.shape

torch.Size([1114481, 512])

In [8]:
labels.shape

torch.Size([1114481, 512])

In [9]:
encodings = {
    
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [10]:
encodings

{'input_ids': tensor([[    0, 30600,   309,  ...,     1,     1,     1],
         [    0,   311,   264,  ...,   265,  3214,     2],
         [    0,     4,   267,  ...,     1,     1,     1],
         ...,
         [    0,     4,   267,  ...,     1,     1,     1],
         [    0,     4,   267,  ...,   399,   279,     2],
         [    0,     4,   268,  ...,   283,   276,     2]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([[    0, 30600,   309,  ...,     1,     1,     1],
         [    0,   311,   264,  ...,   265,  3214,     2],
         [    0,  1116,   267,  ...,     1,     1,     1],
         ...,
         [    0,   899,   267,  ...,     1,     1,     1],
         [    0,   899,   267,  ...,   399,   279,     2],
         [    0,   293,   268,  ...,   283,   276,    

In [11]:
class Dataset (torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self, i):
        return{key: tensor[i] for key, tensor in self.encodings.items()}

In [12]:
dataset = Dataset(encodings)

In [14]:
dataset

In [13]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

In [14]:
dataloader.__len__()

278621

In [16]:
dataloader

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [22]:
model= AutoModelForMaskedLM.from_pretrained('C:/Bangla_dude/checkpoint-2000', return_dict=True).to(device)

In [23]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [24]:
from transformers import AdamW

In [25]:
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# log_dir = "logs/"
# writer = SummaryWriter(log_dir=log_dir)

# # Create an EventAccumulator object to load the log file
# event_acc = EventAccumulator(log_dir)
# event_acc.Reload()

# # Get the scalar summaries from the log file
# tags = event_acc.Tags()['scalars']
# for tag in tags:
#     events = event_acc.Scalars(tag)
#     for event in events:
#         writer.add_scalar(tag, event.value, event.step)


In [ ]:
# writer

In [ ]:
# # set the second log directory
# log_dir_2 = "logs/2"
# writer_2 = SummaryWriter(log_dir=log_dir_2)

In [27]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard import SummaryWriter
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

log_dir = "logs/"
writer = SummaryWriter(log_dir=log_dir)

In [ ]:


# Define the hyperparameters
per_gpu_train_batch_size = 24
per_gpu_eval_batch_size = 4
gradient_accumulation_steps = 2 # Number of updates steps to accumulate before performing a backward/update pass
gradient_accumulation_steps = 1
learning_rate = .01
weight_decay = 0.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 3
max_steps = -1
warmup_steps = 0
logging_steps = 1000
save_steps = 5000

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=max_steps)

# Define the training loop
global_step = 0
best_loss = float('inf')
for epoch in range(num_train_epochs):
    print(f'Epoch {epoch + 1}/{num_train_epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss /= gradient_accumulation_steps # Normalize the loss because it is averaged
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        
        # Update the parameters
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            
            # Log training loss
            epoch_loss += loss.item()
            if global_step % logging_steps == 0:
                print(f'Step {global_step}: Training loss: {epoch_loss/logging_steps}')
                writer.add_scalar('Training loss', epoch_loss/logging_steps, global_step)
                epoch_loss = 0
            
            # Save model checkpoint
            if global_step % save_steps == 0:
                output_dir = f'./checkpoints/checkpoint-{global_step}'
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model.save_pretrained(output_dir)
                print(f'Saved model checkpoint to {output_dir}')
                
                    
                model.train()
                
            if max_steps > 0 and global_step > max_steps:
                break
        
    if max_steps > 0 and global_step > max_steps:
        break
    
writer.close()


Epoch 1/3


  0%|          | 0/278621 [00:00<?, ?it/s]

Step 1000: Training loss: 10.90858830833435
Step 2000: Training loss: 10.908246829032898
Step 3000: Training loss: 10.908221113204956
Step 4000: Training loss: 10.908805342674256
Step 5000: Training loss: 10.90810200023651
Saved model checkpoint to ./checkpoints/checkpoint-5000


KeyboardInterrupt: 

In [28]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

# log_dir = "logs/"
# writer = SummaryWriter(log_dir=log_dir)

# Define the hyperparameters
per_gpu_train_batch_size = 32
per_gpu_eval_batch_size = 4
gradient_accumulation_steps = 100
learning_rate = 1e-5
weight_decay = 0.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 3
max_steps = -1
warmup_steps = 10000
logging_steps = 1
save_steps = 50

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=max_steps)

# Define the training loop
global_step = 0
best_loss = float('inf')
for epoch in range(num_train_epochs):
    print(f'Epoch {epoch + 1}/{num_train_epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        
        # Update the parameters
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            
            # Log training loss
            epoch_loss += loss.item()
            if global_step % logging_steps == 0:
                print(f'Step {global_step}: Training loss: {epoch_loss/logging_steps}')
                writer.add_scalar('Training loss', epoch_loss/logging_steps, global_step)
                epoch_loss = 0
            
            # Save model checkpoint
            if global_step % save_steps == 0:
                output_dir = f'./checkpoints/checkpoint-{global_step}'
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model.save_pretrained(output_dir)
                print(f'Saved model checkpoint to {output_dir}')
                
                    
                model.train()
                
            if max_steps > 0 and global_step > max_steps:
                break
        
    if max_steps > 0 and global_step > max_steps:
        break
    
writer.close()


Epoch 1/3


  0%|          | 0/278621 [00:00<?, ?it/s]

Step 1: Training loss: 4.728098392486572
Step 2: Training loss: 2.9990234375
Step 3: Training loss: 3.944324493408203
Step 4: Training loss: 2.5557453632354736
Step 5: Training loss: 3.993995428085327
Step 6: Training loss: 2.436546564102173
Step 7: Training loss: 2.5162978172302246
Step 8: Training loss: 4.281270503997803
Step 9: Training loss: 3.0095937252044678
Step 10: Training loss: 3.233635663986206
Step 11: Training loss: 2.568081855773926
Step 12: Training loss: 2.4903323650360107
Step 13: Training loss: 2.6215434074401855
Step 14: Training loss: 3.314723014831543
Step 15: Training loss: 4.425722122192383
Step 16: Training loss: 4.745405197143555
Step 17: Training loss: 3.9612879753112793
Step 18: Training loss: 4.351767539978027
Step 19: Training loss: 3.066232681274414
Step 20: Training loss: 3.0739309787750244
Step 21: Training loss: 3.4359536170959473
Step 22: Training loss: 2.8286311626434326
Step 23: Training loss: 2.5409109592437744
Step 24: Training loss: 3.837572097778

  0%|          | 0/278621 [00:00<?, ?it/s]

Step 2787: Training loss: 2.6262729167938232
Step 2788: Training loss: 1.904886245727539
Step 2789: Training loss: 1.6338891983032227
Step 2790: Training loss: 1.8357161283493042
Step 2791: Training loss: 1.494949221611023
Step 2792: Training loss: 1.9159682989120483
Step 2793: Training loss: 1.199481725692749
Step 2794: Training loss: 1.3795397281646729
Step 2795: Training loss: 1.2983553409576416
Step 2796: Training loss: 1.6555415391921997
Step 2797: Training loss: 1.9164968729019165
Step 2798: Training loss: 2.6130330562591553
Step 2799: Training loss: 2.732544422149658
Step 2800: Training loss: 1.8473492860794067
Saved model checkpoint to ./checkpoints/checkpoint-2800
Step 2801: Training loss: 1.962080478668213
Step 2802: Training loss: 1.3498528003692627
Step 2803: Training loss: 1.2391992807388306
Step 2804: Training loss: 1.5613069534301758
Step 2805: Training loss: 1.7448294162750244
Step 2806: Training loss: 1.6199204921722412
Step 2807: Training loss: 1.8577125072479248
Step

In [24]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

log_dir = "logs/"
writer = SummaryWriter(log_dir=log_dir)

# Define the hyperparameters
per_gpu_train_batch_size = 4
per_gpu_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 1
weight_decay = 0.0
adam_epsilon = 1
max_grad_norm = 1.0
num_train_epochs = 3
max_steps = -1
warmup_steps = 0
logging_steps = 1000
save_steps = 5000

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=max_steps)

# Define the training loop
global_step = 0
best_loss = float('inf')
for epoch in range(num_train_epochs):
    print(f'Epoch {epoch + 1}/{num_train_epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        
        # Update the parameters
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            
            # Log training loss
            epoch_loss += loss.item()
            if global_step % logging_steps == 0:
                print(f'Step {global_step}: Training loss: {epoch_loss/logging_steps}')
                writer.add_scalar('Training loss', epoch_loss/logging_steps, global_step)
                epoch_loss = 0
            
            # Save model checkpoint
            if global_step % save_steps == 0:
                output_dir = f'./checkpoints/checkpoint-{global_step}'
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model.save_pretrained(output_dir)
                print(f'Saved model checkpoint to {output_dir}')
                
                    
                model.train()
                
            if max_steps > 0 and global_step > max_steps:
                break
        
    if max_steps > 0 and global_step > max_steps:
        break
    
writer.close()


Epoch 1/3


  0%|          | 0/278621 [00:00<?, ?it/s]

Step 1000: Training loss: 10.909353368759156
Step 2000: Training loss: 10.909596610069276
Step 3000: Training loss: 10.908786114692688
Step 4000: Training loss: 10.90892912387848
Step 5000: Training loss: 10.909625336647034
Saved model checkpoint to ./checkpoints/checkpoint-5000


KeyboardInterrupt: 

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

log_dir = "logs/"
log_file = "my_log_file.log"

# Specify the log file name in the log_dir directory
log_path = os.path.join(log_dir, log_file)

# Create the SummaryWriter object with the specified log file
writer = SummaryWriter(log_dir=log_dir, filename_suffix=log_file)


In [22]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
import os

log_dir = "logs/"

try:
    # Create the logging directory if it doesn't exist
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Create the SummaryWriter object
    writer = SummaryWriter(log_dir=log_dir)

except Exception as e:
    print(f"Error creating SummaryWriter object: {e}")
    writer = None


In [19]:
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter

log_dir = "logs/"
writer = SummaryWriter(log_dir=log_dir)

# Define the hyperparameters
per_gpu_train_batch_size = 4
per_gpu_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 1e-3
weight_decay = 0.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 3
max_steps = -1
warmup_steps = 0
logging_steps = 100
save_steps = 5000

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=max_steps)

# Define the training loop
global_step = 0
best_loss = float('inf')
for epoch in range(num_train_epochs):
    print(f'Epoch {epoch + 1}/{num_train_epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        
        # Update the parameters
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            
            # Log training loss
            epoch_loss += loss.item()
            if global_step % logging_steps == 0:
                print(f'Step {global_step}: Training loss: {epoch_loss/logging_steps}')
                epoch_loss = 0
            
            # Save model checkpoint
            if global_step % save_steps == 0:
                output_dir = f'./checkpoints/checkpoint-{global_step}'
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model.save_pretrained(output_dir)
                print(f'Saved model checkpoint to {output_dir}')
                
                    
                model.train()
                
            if max_steps > 0 and global_step > max_steps:
                break
        
    if max_steps > 0 and global_step > max_steps:
        break
    



Epoch 1/3


  0%|          | 0/278621 [00:00<?, ?it/s]

Step 100: Training loss: 10.907888774871827
Step 200: Training loss: 10.903826532363892
Step 300: Training loss: 10.904981193542481
Step 400: Training loss: 10.90920166015625
Step 500: Training loss: 10.906534976959229
Step 600: Training loss: 10.909966363906861
Step 700: Training loss: 10.910112705230713
Step 800: Training loss: 10.905688705444335
Step 900: Training loss: 10.909560213088989
Step 1000: Training loss: 10.904947700500488
Step 1100: Training loss: 10.909452152252197
Step 1200: Training loss: 10.911711168289184
Step 1300: Training loss: 10.905303325653076
Step 1400: Training loss: 10.907367601394654
Step 1500: Training loss: 10.911247653961182
Step 1600: Training loss: 10.903938703536987
Step 1700: Training loss: 10.910807628631591
Step 1800: Training loss: 10.907160120010376
Step 1900: Training loss: 10.911795253753661
Step 2000: Training loss: 10.907452135086059
Step 2100: Training loss: 10.905585432052613
Step 2200: Training loss: 10.908268795013427
Step 2300: Training 

KeyboardInterrupt: 

In [25]:
from tqdm.auto import tqdm
from transformers import RobertaForMaskedLM, RobertaTokenizerFast
import torch
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
from tqdm import tqdm

In [26]:
model_name= 'C:/Bangla_dude/checkpoints/checkpoint-5000'

In [27]:
model = RobertaForMaskedLM.from_pretrained(model_name)
tokenizer = RobertaTokenizerFast.from_pretrained('bangla_tokenizer')

In [28]:
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoTokenizer
import torch

# Define the hyperparameters
per_gpu_train_batch_size = 4
per_gpu_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 1e-3
weight_decay = 0.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 3
max_steps = -1
warmup_steps = 0
logging_steps = 100
save_steps = 5000

# Load the saved checkpoint
checkpoint_dir = './checkpoints/checkpoint-5000'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=max_steps)


# Define the training loop
global_step = 5000
best_loss = float('inf')
for epoch in range(num_train_epochs):
    print(f'Epoch {epoch + 1}/{num_train_epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        
        # Update the parameters
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            
            # Log training loss
            epoch_loss += loss.item()
            if global_step % logging_steps == 0:
                print(f'Step {global_step}: Training loss: {epoch_loss/logging_steps}')
                epoch_loss = 0
            
            # Save model checkpoint
            if global_step % save_steps == 0:
                output_dir = f'./checkpoints/checkpoint-{global_step}'
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model.save_pretrained(output_dir)
      
            if max_steps > 0 and global_step > max_steps:
                break
        
    if max_steps > 0 and global_step > max_steps:
        break
    


Some weights of the model checkpoint at ./checkpoints/checkpoint-5000 were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./checkpoints/checkpoint-5000 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifi

Epoch 1/3


  0%|          | 0/278621 [00:04<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [45]:
# Define the training loop
epochs = 5
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    model.train()
    epoch_loss = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        # Move the inputs and labels to the device (GPU or CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Update the parameters
        optimizer.step()
        scheduler.step()
        
        epoch_loss += loss.item()
        
    print(f'Training loss: {epoch_loss/len(dataloader)}')

Epoch 1/5


  0%|          | 785/278621 [04:01<23:47:11,  3.24it/s]


KeyboardInterrupt: 

In [ ]:
epoch =3

In [35]:
loop = tqdm(dataloader, leave=True)
for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=mask, labels=labels)

    loss = outputs.loss
    loss.backward()
    optim.step()
    loop.set_description(f'Loss: {loss.item()}')
    loop.set_postfix(loss=loss.item())
    

  0%|          | 0/278621 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (4) must match the size of tensor b (1114481) at non-singleton dimension 0